# Sistema de Análise de Ondas de Calor em Python
Desenvolva um sistema que identifica e analisa ondas de calor usando dados históricos de temperatura. O código deve:
- Detectar períodos onde a temperatura máxima excede o percentil 90 por 3+ dias consecutivos
- Calcular intensidade, duração e frequência das ondas de calor
- Gerar mapas de calor temporal, gráficos de tendências decadais e relatórios de impacto climático
- Implementar algoritmos de detecção baseados em índices como HWMId (Heat Wave Magnitude Index daily)

## Importação das bibliotecas

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import calendar
from datetime import datetime, timedelta


## Configurações iniciais

In [3]:
plt.style.use('ggplot')
sns.set_palette("YlOrRd")

## Desenvolvimento da classe

In [ ]:
class HeatWaveAnalyzer:
    def __init__(self, temp_data, threshold_percentile=90, min_consecutive_days=3):
        """
        Inicializa o analisador de ondas de calor.
        
        Args:
            temp_data (DataFrame): Dados de temperatura com colunas 'date' e 'temp_max'
            threshold_percentile (int): Percentil para definir o limiar de onda de calor (padrão: 90)
            min_consecutive_days (int): Mínimo de dias consecutivos para considerar onda de calor (padrão: 3)
        """
        self.data = temp_data.copy()
        self.threshold_percentile = threshold_percentile
        self.min_consecutive_days = min_consecutive_days
        self.heat_waves = None
        self.climate_report = None

        # Pré-processamento
        self._preprocess_data()
    
    def _preprocess_data(self):
        """Prepara os dados para analise."""
        # Garante que a data está no formato correto
        self.data['date'] = pd.to_datetime(self.data['date'])
        self.data = self.data.sort_values('date').reset_index(drop=True)

        # Adiciona colunas auxiliares
        self.data['year'] = self.data['date'].dt.year
        self.data['month'] = self.data['date'].dt.month
        self.data['day'] = self.data['date'].dt.day
        self.data['day_of_year'] = self.data['date'].dt.dayofyear

        # Calcula o limiar de temperatura para cada dia do ano (percentil movel)
        self._calculate_thresholds()
    
    def _calculate_thresholds(self, window_size=15):
        """
        Calcula o limiar de temperatura para cada dia do ano usando uma janela móvel.
        
        Args:
            window_size (int): Tamanho da janela em dias para calcular o percentil (padrão: 15)
        """
        thresholds = []
        for day in range(1, 367): # para cada dia do ano
            windows_start = max(1, day - window_size)
            windows_end = min(366, day + window_size)

            # filtra os dados na janela
            window_data = self.data[
                (self.data['day'] >= windows_start) &
                (self.data['day'] <= windows_end)
            ]['temp_max']

            if not window_data.empty:
                threshold = np.percentile(window_data, self.threshold_percentile)
                thresholds.append((day, threshold))
            else:
                thresholds.append((day, np.nan))
        
        #Cria um DataFrame com os limiares por dia do ano
        self.data = self.data.merge(self.threshold_df, on='day', how='left')
    
    def detect_heat_waves(self):
        """Detecta períodos de onda de calor com base nos critérios definidos."""
        # Identifica dias acima do limiar
        self.data['above_threshold'] = self.data['temp_max'] > self.data['threshold']
        
        # Identifica períodos consecutivos
        self.data['group'] = (self.data['above_threshold'] != 
                             self.data['above_threshold'].shift()).cumsum()
        
        # Agrupa os períodos consecutivos
        heat_wave_candidates = self.data[self.data['above_threshold']].groupby('group')
        
        # Filtra apenas os períodos com duração mínima
        heat_waves = []
        for name, group in heat_wave_candidates:
            if len(group) >= self.min_consecutive_days:
                start_date = group['date'].min()
                end_date = group['date'].max()
                duration = (end_date - start_date).days + 1
                max_temp = group['temp_max'].max()
                mean_temp = group['temp_max'].mean()
                intensity = group['temp_max'].sum() - group['threshold'].sum()
                
                # Calcula o HWMId (Heat Wave Magnitude Index daily)
                hwmid = self._calculate_hwmid(group)
                
                heat_waves.append({
                    'start_date': start_date,
                    'end_date': end_date,
                    'duration': duration,
                    'max_temp': max_temp,
                    'mean_temp': mean_temp,
                    'intensity': intensity,
                    'hwmid': hwmid,
                    'year': start_date.year
                })
        
        self.heat_waves = pd.DataFrame(heat_waves)
        return self.heat_waves
    
    def _calculate_hwmid(self, heat_wave_data):
        """
        Calcula o Heat Wave Magnitude Index daily (HWMId) para uma onda de calor.
        
        HWMId é definido como o máximo da soma das temperaturas acima do limiar 
        durante a onda de calor.
        """
        excess_temp = heat_wave_data['temp_max'] - heat_wave_data['threshold']
        hwmid = excess_temp.sum()
        return hwmid
    def generate_climate_report(self):
        """Gera um relatório climático com estatísticas das ondas de calor."""
        if self.heat_waves is None:
            self.detect_heat_waves()
        
        report = {
            'total_heat_waves': len(self.heat_waves),
            'avg_duration': self.heat_waves['duration'].mean(),
            'max_duration': self.heat_waves['duration'].max(),
            'avg_intensity': self.heat_waves['intensity'].mean(),
            'max_intensity': self.heat_waves['intensity'].max(),
            'avg_hmwid': self.heat_waves['hwmid'].mean(),
            'max_hmwid': self.heat_waves['hwmid'].max(),
            'annual_frequency': self._calculate_annual_frequency(),
            'decadal_trend': self._calculate_decadal_trend(),
            'monthly_distribution': self._calculate_monthly_distribution()
        }
        self.climate_report = report
        return report
       
